In [24]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
def fetch_all_indicators(credentials_path = "connection-123-892e002c2def.json") -> pd.DataFrame:
        """Fetch and combine all technical indicators from BigQuery"""
        credentials = service_account.Credentials.from_service_account_file(credentials_path)
        client = bigquery.Client(credentials=credentials, project="connection-123")

        # Main query joining all indicator tables
        query = """
        WITH base_data AS (
            SELECT
                p.timestamp as date_,
                p.price,
                p.market_cap,
                p.total_volume,
                ma.sma_10,
                ma.sma_20,
                ma.sma_50,
                ema.ema_9,
                ema.ema_12,
                ema.ema_26,
                ema.ema_20,
                ema.ema_50
            FROM `connection-123.signals.bitcoin_price` p
            LEFT JOIN `connection-123.signals.btc_moving_averages` ma ON p.timestamp = ma.date_
            LEFT JOIN `connection-123.signals.btc_ema` ema ON p.timestamp = ema.timestamp
            LEFT JOIN `connection-123.signals.btc_rsi` rsi ON p.timestamp = rsi.timestamp
            LEFT JOIN `connection-123.signals.btc_macd` macd ON p.timestamp = macd.timestamp
            LEFT JOIN `connection-123.signals.btc_bollinger_bands` bb ON p.timestamp = bb.timestamp
            WHERE p.timestamp BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 180 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
            ORDER BY p.timestamp
        )
        SELECT *
        FROM base_data
        WHERE sma_10 IS NOT NULL  -- Ensure we have indicator data
        """

        query_job = client.query(query)
        df = query_job.result().to_dataframe()

        return df

In [12]:
data = fetch_all_indicators()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date_         180 non-null    dbdate 
 1   price         180 non-null    float64
 2   market_cap    180 non-null    float64
 3   total_volume  180 non-null    float64
 4   sma_10        180 non-null    float64
 5   sma_20        180 non-null    float64
 6   sma_50        180 non-null    float64
 7   ema_9         180 non-null    float64
 8   ema_12        180 non-null    float64
 9   ema_26        180 non-null    float64
 10  ema_20        180 non-null    float64
 11  ema_50        180 non-null    float64
dtypes: dbdate(1), float64(11)
memory usage: 17.0 KB


In [20]:
X = data.drop(['price','date_'], axis=1)
y = data['price']

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [27]:
model = sm.OLS(Y_train, X_train)
results = model.fit()

In [28]:
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.398e+05
Date:                Tue, 09 Sep 2025   Prob (F-statistic):          4.06e-306
Time:                        17:02:38   Log-Likelihood:                -783.81
No. Observations:                 144   AIC:                             1590.
Df Residuals:                     133   BIC:                             1622.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1576.4154    117.120     13.460   